# Import relevant libraries

In [1]:
import os
import gdal
import glob

# Translate Lat and Lon files
- Find out information about the lat lon file
- Using the gdal_translate tool convert the lat and lon hdr files into vrt format

In [2]:
latlon_info = gdal.Info(r'D:\Annies_Dissertation\MAIAC_Test\subset\MAIACLatlon.h00v01.hdf')
print(latlon_info)

Driver: HDF4/Hierarchical Data Format Release 4
Files: D:\Annies_Dissertation\MAIAC_Test\subset\MAIACLatlon.h00v01.hdf
Size is 512, 512
Coordinate System is `'
Metadata:
  HDFEOSVersion=HDFEOS_V2.12
Subdatasets:
  SUBDATASET_1_NAME=HDF4_EOS:EOS_GRID:"D:\Annies_Dissertation\MAIAC_Test\subset\MAIACLatlon.h00v01.hdf":latlon:lat
  SUBDATASET_1_DESC=[1200x1200] lat latlon (32-bit floating-point)
  SUBDATASET_2_NAME=HDF4_EOS:EOS_GRID:"D:\Annies_Dissertation\MAIAC_Test\subset\MAIACLatlon.h00v01.hdf":latlon:lon
  SUBDATASET_2_DESC=[1200x1200] lon latlon (32-bit floating-point)
Corner Coordinates:
Upper Left  (    0.0,    0.0)
Lower Left  (    0.0,  512.0)
Upper Right (  512.0,    0.0)
Lower Right (  512.0,  512.0)
Center      (  256.0,  256.0)



In [3]:
gdal.Translate(r"D:\Annies_Dissertation\MAIAC_Test\subset\h00v01_lat.vrt", r'HDF4_EOS:EOS_GRID:"D:\Annies_Dissertation\MAIAC_Test\subset\MAIACLatlon.h00v01.hdf":latlon:lat', format = 'VRT' )

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000000A371161570> >

In [4]:
gdal.Translate(r"D:\Annies_Dissertation\MAIAC_Test\subset\h00v01_lon.vrt", r'HDF4_EOS:EOS_GRID:"D:\Annies_Dissertation\MAIAC_Test\subset\MAIACLatlon.h00v01.hdf":latlon:lon', format = 'VRT' )

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000000A3711614E0> >

# Warp the AOT images
- get a list of names of HDF files
- for each file name in list write data.vrt (change file name in data.vrt for each of the files)
    - use os.system or os.path to write a 'data.vrt' text file in xml language 
    - open("data.vrt")
- run gdal_warp and rename projected*.tif

# Get a list of files

In [5]:
filenames = glob.glob(r'D:\Annies_Dissertation\MAIAC_Test\subset\????\MAIACAAOT*.hdf')
print(filenames)

['D:\\Annies_Dissertation\\MAIAC_Test\\subset\\2012\\MAIACAAOT.h00v01.20120011310.hdf', 'D:\\Annies_Dissertation\\MAIAC_Test\\subset\\2012\\MAIACAAOT.h00v01.20120011450.hdf', 'D:\\Annies_Dissertation\\MAIAC_Test\\subset\\2012\\MAIACAAOT.h00v01.20120021215.hdf', 'D:\\Annies_Dissertation\\MAIAC_Test\\subset\\2012\\MAIACAAOT.h00v01.20120021355.hdf', 'D:\\Annies_Dissertation\\MAIAC_Test\\subset\\2012\\MAIACAAOT.h00v01.20120031300.hdf', 'D:\\Annies_Dissertation\\MAIAC_Test\\subset\\2012\\MAIACAAOT.h00v01.20120031440.hdf', 'D:\\Annies_Dissertation\\MAIAC_Test\\subset\\2012\\MAIACAAOT.h00v01.20120041340.hdf', 'D:\\Annies_Dissertation\\MAIAC_Test\\subset\\2012\\MAIACAAOT.h00v01.20120051245.hdf', 'D:\\Annies_Dissertation\\MAIAC_Test\\subset\\2012\\MAIACAAOT.h00v01.20120051425.hdf', 'D:\\Annies_Dissertation\\MAIAC_Test\\subset\\2012\\MAIACAAOT.h00v01.20120061150.hdf', 'D:\\Annies_Dissertation\\MAIAC_Test\\subset\\2013\\MAIACAAOT.h00v01.20130011145.hdf', 'D:\\Annies_Dissertation\\MAIAC_Test\\subs

# Create geoloc.vrt

In [6]:
xml = """<VRTDataset rasterXSize="1200" rasterYSize="1200">
  <Metadata domain="GEOLOCATION">
     <mdi key="X_DATASET">h00v01_lon.vrt</mdi>
     <mdi key="X_BAND">1</mdi>
     <mdi key="Y_DATASET">h00v01_lat.vrt</mdi>
     <mdi key="Y_BAND">1</mdi>
     <mdi key="PIXEL_OFFSET">0</mdi>
     <mdi key="LINE_OFFSET">0</mdi>
     <mdi key="PIXEL_STEP">1</mdi>
     <mdi key="LINE_STEP">1</mdi>
  </Metadata>
  <VRTRasterBand dataType="Float32" band="1">
    <ColorInterp>Gray</ColorInterp>
    <SimpleSource>
      <SourceFilename relativeToVRT="0">HDF4_EOS:EOS_GRID:"{fname}":grid1km:Optical_Depth_Land</SourceFilename>
      <SourceBand>1</SourceBand>
      <SourceProperties RasterXSize="1200" RasterYSize="1200" DataType="Float32" BlockXSize="1200" BlockYSize="1200" />
      <SrcRect xOff="0" yOff="0" xSize="1200" ySize="1200" />
      <DstRect xOff="0" yOff="0" xSize="1200" ySize="1200" />
    </SimpleSource>
  </VRTRasterBand>
</VRTDataset>
"""

# Function which writes geoloc.vrt for each filename

In [7]:
def geoloc(fname):
    with open(r'D:\Annies_Dissertation\MAIAC_Test\subset\geoloc_test.vrt', 'w') as myfile:
        myfile.write(xml.format(fname=fname))

# Funcrion that warps each image and produces a projected image

In [8]:
def warp(fname):
    (gdal.Warp(r'{fname}_proj.tif'.format(fname=fname),
               r'D:\Annies_Dissertation\MAIAC_Test\subset\geoloc_test.vrt',
               geoloc = True, srcSRS = 'EPSG:4326', dstSRS = 'EPSG:27700'))

# For statement which loops over each file and produces the geoloc.vrt and projects the image

In [9]:
for filename in filenames:
    print('Geolocating {fname}'.format(fname=filename))
    geoloc(filename)
    print('Warping {fname}'.format(fname=filename))
    warp(filename)
    

Geolocating D:\Annies_Dissertation\MAIAC_Test\subset\2012\MAIACAAOT.h00v01.20120011310.hdf
Warping D:\Annies_Dissertation\MAIAC_Test\subset\2012\MAIACAAOT.h00v01.20120011310.hdf
Geolocating D:\Annies_Dissertation\MAIAC_Test\subset\2012\MAIACAAOT.h00v01.20120011450.hdf
Warping D:\Annies_Dissertation\MAIAC_Test\subset\2012\MAIACAAOT.h00v01.20120011450.hdf
Geolocating D:\Annies_Dissertation\MAIAC_Test\subset\2012\MAIACAAOT.h00v01.20120021215.hdf
Warping D:\Annies_Dissertation\MAIAC_Test\subset\2012\MAIACAAOT.h00v01.20120021215.hdf
Geolocating D:\Annies_Dissertation\MAIAC_Test\subset\2012\MAIACAAOT.h00v01.20120021355.hdf
Warping D:\Annies_Dissertation\MAIAC_Test\subset\2012\MAIACAAOT.h00v01.20120021355.hdf
Geolocating D:\Annies_Dissertation\MAIAC_Test\subset\2012\MAIACAAOT.h00v01.20120031300.hdf
Warping D:\Annies_Dissertation\MAIAC_Test\subset\2012\MAIACAAOT.h00v01.20120031300.hdf
Geolocating D:\Annies_Dissertation\MAIAC_Test\subset\2012\MAIACAAOT.h00v01.20120031440.hdf
Warping D:\Annies_D